## pdf to markdown

* Azure AI Document Intelligenceでpdfをmarkdown化する

### 設定

In [1]:
!pip install python-dotenv
!pip install azure-ai-documentintelligence


[notice] A new release of pip is available: 24.0 -> 25.0
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.0 -> 25.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import os

from dotenv import load_dotenv
load_dotenv(verbose=True)
dotenv_path = ".env"
load_dotenv(dotenv_path)

from azure.core.credentials import AzureKeyCredential
from azure.ai.documentintelligence import DocumentIntelligenceClient
from azure.ai.documentintelligence.models import AnalyzeDocumentRequest, DocumentContentFormat

### 定数

In [3]:
LIST_COMPANY_NAME = [
    "カゴメ株式会社",
    "グローリー株式会社",
    "サントリーホールディングス株式会社",
    "ハウス食品グループ本社株式会社",
    "パナソニックホールディングス株式会社",
    "全国保証株式会社",
    "日清食品ホールディングス株式会社",
    "日産自動車株式会社",
    "明治ホールディングス株式会社",
    "東急不動産ホールディングス株式会社",
    "東洋エンジニアリング株式会社",
    "株式会社IHI",
    "株式会社キッツ",
    "株式会社クレハ",
    "株式会社メディアドゥ",
    "株式会社モスフードサービス",
    "株式会社4℃ホールディングス",
    "株式会社ライフコーポレーション",
    "株式会社高松コンストラクショングループ",
]

### pdfをmarkdown化

In [4]:
endpoint = os.environ["DI_ENDPOINT"]
api_key = os.environ["DI_API_KEY"]

document_intelligence_client = (
    DocumentIntelligenceClient(
        endpoint=endpoint,
        credential=AzureKeyCredential(api_key),
    )
)

In [5]:
def pdf2md(dic: DocumentIntelligenceClient, pdf_url: str):
    poller = document_intelligence_client.begin_analyze_document(
        "prebuilt-layout",
        AnalyzeDocumentRequest(url_source=pdf_url),
        output_content_format=DocumentContentFormat.MARKDOWN,
    )
    result = poller.result()
    md = result.content

    return md

In [6]:
for company_name in LIST_COMPANY_NAME:
    # 抽出
    pdf_url = f"https://signate-fdua-rag.s3.ap-northeast-1.amazonaws.com/{company_name}.pdf"
    md_content = pdf2md(document_intelligence_client, pdf_url)

    # 保存
    md_file_name = f"../data/markdown/{company_name}.md"
    with open(md_file_name, "w", encoding="utf-8") as md_file:
        md_file.write(md_content)